In [1]:
# Import libraries used 
import pandas as pd
import numpy as np

In [2]:

#Convert json file to a csv file

business_json_path = 'Data/yelp_business.json'

business_df = pd.read_json(business_json_path, lines=True)
business_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [3]:
#### Trying to clean other aspects of data ##############

#### General Cleaning#####
# Initial shape of data
print("Shape of data initially " + str(business_df.shape))

# Number of null values
print("This is the total number of null values in the dataframe " + str(business_df.isna().sum().sum()))

# Drop rows where restaraunt is not open
business_df = business_df[business_df.is_open != 0]

# Drop column 'is_open' as not needed anymore
business_df = business_df.drop(['is_open'], axis=1)

# Filter catagories that contain "Restaurants" and #Food
business_df = business_df[business_df['categories'].str.contains('Restaurants|Food', case =False, na=False)]



print("New shape of data " + str(business_df.shape))

Shape of data initially (150346, 14)
This is the total number of null values in the dataframe 37070
New shape of data (44582, 13)


In [4]:
#Create column that counts the number of null values in each row
business_df["Null Count"] = business_df.isnull().sum(axis=1)

In [5]:
#Get rid of restatruants that do not have there hours available
business_df = business_df.dropna(subset=['hours'])

# Separate the days of the week open from the hours column
business_df = pd.concat([business_df, business_df["hours"].apply(pd.Series)], axis=1)


In [6]:
#Replace all NaN values in Saturday and Sunday to have 0 to represent not being open that day 
business_df[['Saturday','Sunday']] = business_df[['Saturday','Sunday']].replace(np.nan,0)

In [7]:
# Function created to check if restaraunt is open on the weekend
def open_weekend_conditions(x):
    if (x['Saturday'] != 0) & (x['Sunday'] != 0):
        return 1
    else:
        return 0
#Apply Function to new column open_weekend in the dataframe
business_df["open_weekends"] = business_df.apply(open_weekend_conditions,axis=1)

In [8]:
# Is restaruant open on a Monday?
#Replace all null values in Monday to be 0 
business_df['Monday'] = business_df['Monday'].replace(np.nan,0)
# change 0:0 -0:0 to 0s 
#Function to convert monday 0 or 0:0-0:0 values to 0 and the rest be 1 to represent open.
business_df['Monday'] = business_df['Monday'].replace('0:0-0:0',0,regex=True)

#Function to convert open to 1 and 0 for restaraunts that are not open on monday
def open_on_monday(x):
    if (x['Monday'] != 0):
        return 1
    else:
        return 0

#Apply the function to create open_Monday column
business_df["open_monday"] = business_df.apply(open_on_monday,axis=1)

## Drop days of the week columns not needed anymore 
business_df =business_df.drop(business_df.iloc[:,14:21],axis=1)

## Drop hours column as not needed anymore
business_df = business_df.drop('hours', axis=1)

In [9]:
business_df = business_df.dropna(subset=['categories'])

In [10]:
#Delete rows that do no have any attributes information
business_df = business_df.dropna(subset=['attributes'])
## Extract the attributes column to its own column
business_df = pd.concat([business_df, business_df["attributes"].apply(pd.Series)], axis=1)
business_df.head()
## Make all attributes columns that are nan to 0 
business_df= business_df.fillna(0)

# Make the 'None' type values into 0 throughout the dataframe

# Replace String using apply() function with lambda.
business_df = business_df.apply(lambda x: x.replace({"none": 0, "None": 0,"False": 0,"u'none'": 0," u 'no'": 0,'no': 0,}, regex=True))
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,BestNights,BYOB,Corkage,BYOBCorkage,AcceptsInsurance,RestaurantsCounterService,Open24Hours,AgesAllowed,DietaryRestrictions,HairSpecializesIn
3,MTSW4McQd7CbVtyjqoe9mw,0,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0,0,0,0,0,0,0,0,0,0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,0,0,0,0,0,0,0,0,0,0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,0,0,0,0,0,0,0,0,0,0
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,0,0,0,0,0,0,0,0,0,0
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Track columns that can be taken out AgesAllowed HairSpecializesIn AcceptsInsurance BYOBCorkage GoodForDancing BestNights BYOB ByAppointmentOnly BusinessParking RestaurantsAttire Ambience CoatCheck GoodForMeal BusinessAcceptsBitcoin 
business_df = business_df.drop(['AgesAllowed','HairSpecializesIn', 'Music','AcceptsInsurance','BYOBCorkage', 'Corkage','GoodForDancing','BestNights','BYOB','ByAppointmentOnly','BusinessParking','RestaurantsAttire','Ambience', 'CoatCheck', 'GoodForMeal', 'BusinessAcceptsBitcoin'], axis = 1)

In [12]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,RestaurantsTableService,RestaurantsGoodForGroups,HasTV,HappyHour,DriveThru,NoiseLevel,Smoking,RestaurantsCounterService,Open24Hours,DietaryRestrictions
3,MTSW4McQd7CbVtyjqoe9mw,0,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0,0,0,0,0,0,0,0,0,0
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,...,0,0,0,0,0,0,0,0,0,0
5,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,...,0,True,True,0,True,0,0,0,0,0
9,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,...,0,0,True,0,True,0,0,0,0,0
11,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,...,0,0,0,0,0,0,0,0,0,0


In [15]:
business_df['Null Count'].value_counts()

0    39448
Name: Null Count, dtype: int64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fcd92ccc-2af6-49a1-bc78-ae9f0a350628' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>